# Record Size Limit
For this test, to keep the record generation code reasonable, we configure namespace _test_ on the server **max-record-size 1K**.)_
dynamically using _asadm_ or changing the server configuration.

```
asadm --enable -e "manage config namespace test param max-record-size to 1K" -h "127.0.0.1"
```

Alternatively, you can use the terminal to edit:
```
sudo vi /etc/aerospike/aerospike.conf  --> change namespace test: max-record-size 1K
sudo service aerospike stop
sudo rm /opt/aeropsike/data/test.dat
sudo service aerospike start
```

### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [12]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [13]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [14]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

In [15]:
%sh asadm --enable -e "manage config namespace test param max-record-size to 1K" -h "127.0.0.1"

####  Java Client Imports and Test code

In [16]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpReadFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.exp.ListExp;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

import java.util.Random;

//Write block size is configured at 1K
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
Key key = new Key("test", "testset", "key1");
WritePolicy wPolicy = new WritePolicy();
Bin bint = new Bin("bint", Value.get(1)); //int bin
Bin bstr = new Bin("bstr", Value.get("test--test")); //string bin
Bin bstr2 = new Bin("b2", Value.get("-1234567"));
Bin bstr3 = new Bin("b3", Value.get("-1234567"));  //will go to 1024
//Bin bstr3 = new Bin("b3", Value.get("-1234567")); //will push to record-too-big
Bin big1 = new Bin("big1", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));
Bin big2 = new Bin("big2", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));
Bin big3 = new Bin("big3", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));
Bin big4 = new Bin("big4", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));
Bin big5 = new Bin("big5", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));
Bin big6 = new Bin("big6", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));
Bin big7 = new Bin("big7", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));
Bin big8 = new Bin("big8", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x"));

//Add another character and you will push to Record Too Big error.
//Bin big8 = new Bin("big8", Value.get("0123456789x----x0123456789x----x0123456789x----x0123456789x----x1"));

client.delete(null,key);
client.put(wPolicy, key, bint, bstr, bstr2, big1, big2, big3, big4, big5, big6, big7, big8);
System.out.println("Record: "+ client.get(null, key)); //check record got inserted

//Using expressions to read record metadata in PI that has size of record on device
Expression recSizeExp = Exp.build(Exp.deviceSize());
Record record = client.operate( wPolicy, key,   
          ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT) 
         );
System.out.println("Record Size in bytes: " + record.getValue("size"));

// 35 + 4 (has ttl) + 1 + 7 (testset) + 1 + [(1+4)+(2+4)] + [(1+4)+(5+11)] + [1+2+5+9] = 89 --> 96

client.operate(wPolicy, key, Operation.put(Bin.asNull("b2")), Operation.put(bstr3));
System.out.println("Record: "+ client.get(null, key)); //check record got inserted
record = client.operate( wPolicy, key,   
          ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT) 
         );
System.out.println("Record Size in bytes after b2 is null: " + record.getValue("size"));


Record: (gen:1),(exp:450949345),(bins:(bint:1),(bstr:test--test),(b2:-1234567),(big1:0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x),(big2:0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x),(big3:0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x),(big4:0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x),(big5:0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x),(big6:0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x),(big7:0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x0123456789x----x),(big8:0123456789x----x0123456789x----x0123456789x----x0123456789x----x))
Record S

# Cleanup

In [17]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"